In [3]:
# Copyright 2024 D-Wave
# Licensed under the Apache License, Version 2.0

import ast
import dimod
import numpy as np
import time


"""def TTS(p, t=1):
    ret = t * np.max([np.ones_like(p), np.abs(np.log(0.01) / np.log(1 - p))], axis=0)
    return ret"""

def TTS(p, t):
    """
    Calcula el TTS de forma segura para valores escalares o arrays.
    Asigna infinito si p = 0, evita log(0) y divisiones por cero.
    """
    p = np.asarray(p)
    t = np.asarray(t)
    eps = 1e-10  # para evitar log(0)

    # Evita problemas de log(0)
    p_safe = np.clip(p, eps, 1 - eps)
    tts = t * np.maximum(1, np.abs(np.log(0.01) / np.log(1 - p_safe)))

    # Si es escalar
    if p.shape == ():
        return np.inf if p == 0 else tts.item()

    # Si es array
    tts[p == 0] = np.inf
    return tts






def parse_problems(_problems):
    linear = _problems[0][0]
    linear.update(_problems[0][1])
    quadratic = _problems[1][0]
    quadratic.update(_problems[1][1])
    quadratic.update(_problems[1][2])
    cubic = _problems[2][0]
    return linear, quadratic, cubic


def descend_to_local_minimum(samples, linear, quadratic, cubic, max_sweeps=5):
    _samples = samples.copy()
    for isample, sample in enumerate(_samples):
        for sweep in range(max_sweeps):
            improved = False
            vp = np.random.permutation(len(samples[0]))
            for v in vp:
                ene = eval_cubic_energies(linear, quadratic, cubic, sample)
                temp = sample.copy()
                temp[v] *= -1
                newene = eval_cubic_energies(linear, quadratic, cubic, temp)
                if newene < ene:
                    improved = True
                    sample[v] *= -1
            if not improved:
                break
    return _samples


def embed_multiple_copies(_bqm, _multiple_embeddings):
    embedded_bqm = dimod.BinaryQuadraticModel(vartype=dimod.SPIN)
    for emb in _multiple_embeddings:
        for v in emb:
            embedded_bqm.add_variable(v)
    for v, bias in _bqm.linear.items():
        for iemb in range(len(_multiple_embeddings)):
            embedded_bqm.add_linear(_multiple_embeddings[iemb][v], bias)
    for (u, v), bias in _bqm.quadratic.items():
        for iemb in range(len(_multiple_embeddings)):
            embedded_bqm.add_quadratic(_multiple_embeddings[iemb][u], _multiple_embeddings[iemb][v], bias)
    return embedded_bqm


def eval_cubic_energies(_linear, _quadratic, _cubic, _samples):
    _samples = np.atleast_2d(_samples)
    linmat = np.zeros((1, _samples.shape[1]), dtype=int)
    quadmat = np.zeros((3, len(_quadratic)), dtype=int)
    cubmat = np.zeros((4, len(_cubic)), dtype=int)
    linmat[0, :] = np.array([_linear[key] for key in range(len(_linear))])
    quadmat[0:2, :] = np.asarray(list(_quadratic.keys())).T
    quadmat[2, :] = np.array(list(_quadratic.values()))
    cubmat[0:3, :] = np.asarray(list(_cubic.keys())).T
    cubmat[3, :] = np.array(list(_cubic.values()))
    ret = np.sum(_samples * linmat, axis=1)
    temp = _samples[:, quadmat[0, :]] * _samples[:, quadmat[1, :]] * quadmat[2, :]
    ret += np.sum(temp, axis=1)
    temp = _samples[:, cubmat[0, :]] * _samples[:, cubmat[1, :]] * _samples[:, cubmat[2, :]] * cubmat[3, :]
    ret += np.sum(temp, axis=1)
    return np.asarray(ret)


def make_ising_instance(_linear, _quadratic, _cubic):
    bqm = dimod.BinaryQuadraticModel(vartype=dimod.SPIN)
    for v in sorted(_linear.keys()):
        bqm.add_variable(v)
        bqm.add_linear(v, _linear[v])
    for u, v in _quadratic.keys():
        bqm.add_quadratic(u, v, _quadratic[u, v])
    for (x, y, z), sign in _cubic.items():
        u = len(bqm)
        v = u + 1
        bqm.add_variable(u)
        bqm.add_variable(v)
        if np.abs(y - z) > 2:
            if options['BETTER_GADGETS']:
                bqm.add_linear(u, 6)
                bqm.add_linear(x, -3)
                bqm.add_linear(y, -3)
                bqm.add_linear(z, 1 * sign)
                bqm.add_quadratic(u, x, -4)
                bqm.add_quadratic(u, y, -4)
                bqm.add_quadratic(u, z, 2 * sign)
                bqm.add_quadratic(v, y, -1)
                bqm.add_quadratic(v, z, -1 * sign)
                bqm.add_quadratic(x, y, 1)
                bqm.add_quadratic(x, z, -1 * sign)
            else:
                bqm.add_linear(z, 1 * sign)
                bqm.add_quadratic(v, y, -1)
                bqm.add_quadratic(v, z, -sign)
                bqm.add_quadratic(u, x, -4)
                bqm.add_quadratic(u, y, -4)
                bqm.add_quadratic(u, z, 2 * sign)
                bqm.add_quadratic(x, z, -sign)
                if sign == 1:
                    bqm.add_linear(u, 6)
                    bqm.add_linear(x, -3)
                    bqm.add_linear(y, -3)
                    bqm.add_quadratic(x, y, 1)
                else:
                    bqm.add_linear(u, 2)
                    bqm.add_linear(x, -1)
                    bqm.add_linear(y, -1)
                    bqm.add_quadratic(x, y, 3)
        else:
            bqm.add_linear(u, -2)
            bqm.add_linear(v, -1)
            bqm.add_linear(x, -1)
            bqm.add_linear(y, -1)
            bqm.add_quadratic(u, x, 2)
            bqm.add_quadratic(u, y, 2)
            bqm.add_quadratic(v, x, 1)
            bqm.add_quadratic(v, y, 1)
            bqm.add_quadratic(v, z, 1 * sign)
            bqm.add_quadratic(u, v, 2)
            bqm.add_quadratic(x, y, 1)
    return bqm


def collate_and_prune_samples(_qpu_samples, num_qubits, num_spins):
    full_qpu_samples = np.zeros((len(_qpu_samples), num_qubits))
    full_qpu_samples[:, np.unique(multiple_embeddings)] = dimod.as_samples(_qpu_samples)[0]
    collated_samples = [full_qpu_samples[:, embedding] for embedding in multiple_embeddings]
    return np.reshape(np.asarray(collated_samples), (-1, collated_samples[0].shape[-1]))[:, :num_spins]


# ---- CONFIGURACIÓN ----
options = {
    "RUN_ALL": False,
    "BETTER_GADGETS": True,
    "NUM_REPETITIONS": 3
}

# Usamos el sampler clásico de dimod
sampler = dimod.SimulatedAnnealingSampler()

with open("optimal_solutions/ibm_washington_cubic_instances_minimum_energies.txt", "r") as file:
    gse = ast.literal_eval(file.read())
with open("optimal_solutions/ibm_washington_cubic_instances_maximum_energies.txt", "r") as file:
    maxene = ast.literal_eval(file.read())

seeds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 69] if options["RUN_ALL"] else [5]

data = []
for seed in seeds:
    for loops in range(options["NUM_REPETITIONS"]):
        tic = time.time()

        with open(f"problem_instances/ibm_washington_{seed}.txt", "r") as file:
            problems = ast.literal_eval(file.read())

        linear, quadratic, cubic = parse_problems(problems)
        bqm = make_ising_instance(linear, quadratic, cubic)

        print(f'Instance {seed} constructed. Running simulated annealing...')
        qpu_sampleset = sampler.sample(bqm, num_reads=500)
        print('done. Postprocessing...')

        #samples = np.array([list(s.values()) for s in qpu_sampleset.samples()])
        samples = np.array([list(sample.values())[:len(linear)] for sample in qpu_sampleset])


        energies = eval_cubic_energies(linear, quadratic, cubic, samples)

        wall_time = time.time() - tic
        samples = descend_to_local_minimum(samples, linear, quadratic, cubic)
        pp_energies = eval_cubic_energies(linear, quadratic, cubic, samples)

        time_per_sample = wall_time / len(samples)

        data.append([
            seed,
            np.mean(energies),
            np.min(energies),
            gse[seed],
            np.mean(energies == gse[seed]),
            wall_time,
            wall_time,
            TTS(np.mean(energies == gse[seed]), time_per_sample),
            time_per_sample,
            np.mean(pp_energies == gse[seed]),
            TTS(np.mean(pp_energies == gse[seed]), time_per_sample),
        ])

        print(
            f'\nSEED {seed}: Mean energy {data[-1][1]:.4f}, minimum energy {data[-1][2]}, '
            f'PGS={data[-1][4]:.6f}, mean AR={1 - (data[-1][1] - gse[seed]) / (maxene[seed] - gse[seed]):.4f}')
        print(
            f'     wall time: {data[-1][5]:.3f}s.  Sample time: {data[-1][6] / len(samples):.6f}s.  TTS: {data[-1][7]:.6f}s')
        print(
            f'     postprocessed PGS={data[-1][9]:.6f}, postprocessed TTS: {data[-1][10]:.6f}s')

np.savetxt(
    f'higher_order_spin_glass_results{"_all" * options["RUN_ALL"]}{"_better_gadgets" * options["BETTER_GADGETS"]}.txt',
    np.asarray(data),
    header='seed, mean_ene, min_ene, opt_ene, ground_state_prob, wall_time_s, qpu_time_s, TTS_s, time_per_sample_s, ground_state_prob_pp, TTS_pp_s',
    delimiter=', ', fmt='%d %.6f %d %d %.6f %.6f %.6f %.6f %.6f %.6f %.6f'
)

pgs_pp = np.reshape(np.asarray(data)[:, 9], (len(seeds), -1))
mean_pgs_pp = np.mean(pgs_pp, axis=1)
sampletime = np.reshape(np.asarray(data)[:, 8], (len(seeds), -1))
mean_sampletime = np.mean(sampletime, axis=1)
tts_pp_ms = 1000 * TTS(mean_pgs_pp, mean_sampletime)

print(mean_pgs_pp)
print(tts_pp_ms)


Instance 5 constructed. Running simulated annealing...
done. Postprocessing...

SEED 5: Mean energy -184.4520, minimum energy -198, PGS=0.006000, mean AR=0.9653
     wall time: 240.695s.  Sample time: 0.481391s.  TTS: 368.371315s
     postprocessed PGS=0.018000, postprocessed TTS: 122.048492s
Instance 5 constructed. Running simulated annealing...
done. Postprocessing...

SEED 5: Mean energy -184.3120, minimum energy -196, PGS=0.000000, mean AR=0.9649
     wall time: 236.530s.  Sample time: 0.473060s.  TTS: infs
     postprocessed PGS=0.010000, postprocessed TTS: 216.761107s
Instance 5 constructed. Running simulated annealing...
done. Postprocessing...

SEED 5: Mean energy -184.3080, minimum energy -196, PGS=0.000000, mean AR=0.9649
     wall time: 238.980s.  Sample time: 0.477959s.  TTS: infs
     postprocessed PGS=0.010000, postprocessed TTS: 219.005877s
[0.01266667]
[172490.08511379]
